In [98]:
def composites(wave_ds,precip_ds):    
    wavedict = {}
    
    time_phase = wave_ds["time"].astype("datetime64[D]")
    time_precip = precip_ds["time"].astype("datetime64[D]")

    phases = wave_ds.groupby("phase").groups

    for phase in phases:
        phase_indices = np.where(phase == wave_ds["phase"])[0]
        precip_values = [
            precip_ds.where(time_phase[i] == time_precip).dropna(dim = "time").values[0]
            for i in phase_indices
        ]
        precip_time = [
            precip_ds["time"].where(time_phase[i] == time_precip).dropna(dim = "time").values[0]
            for i in phase_indices
        ]
        wavedict[str(int(phase))] = [precip_time, precip_values]
#         wavedict["time-"+str(int(phase))] = precip_time
    return(wavedict)

def is_seas(month, seas):
    if seas=='jja':
        return (month >= 6) & (month <= 8)
    elif seas=='djf':
        return (month >= 12) | (month <= 2)
    elif seas=='son':
        return (month >= 9) & (month <= 11)
    elif seas=='mam':
        return (month >= 3) & (month <= 5)
        
def placehold(array,fillval):
    if len(array)==0:
        return(fillval)
    else: return(array)

In [99]:
import xarray as xr
import pandas as pd
import numpy as np

seas = 'mam'
if seas=='mam':
    seasname = "SHAutumn"
elif seas=='djf':
    seasname = "SHSummer"

In [100]:
diri = "/Users/fadhlilrizki/Documents/PhD/thesis/dataout/local_wave_phase/"
diro = "/Users/fadhlilrizki/Documents/PhD/thesis/dataout/local_wave_phase/combined_wave/"

In [101]:
regions = ["TopEnd","CapeYork","Kimberley","Queensland"]
latSs   = [-17, -16, -20, -25]
latNs   = [-10, -10, -13.5, -18]
lonRs   = [137, 146, 128, 145]
lonLs   = [129, 142, 120, 140]

code = 1 #regioncode

In [102]:
fw_1 = "local_er_phase_"+seasname+".133.nc"
fw_2 = "local_kelvin_phase_"+seasname+".133.nc"
fw_3 = "local_mrg_phase_"+seasname+".133.nc"
fw_4 = "local_td_phase_"+seasname+".133.nc"
fw_5 = "local_mjo_phase_"+seasname+".133.nc"

In [103]:
dw_1 = xr.open_dataset(diri+fw_1)
dw_2 = xr.open_dataset(diri+fw_2)
dw_3 = xr.open_dataset(diri+fw_3)
dw_4 = xr.open_dataset(diri+fw_4)
dw_5 = xr.open_dataset(diri+fw_5)

dprec = xr.open_dataset("/Users/fadhlilrizki/Documents/PhD/thesis/data/agcd_v1_precip_calib_r025_daily_1981-2020.masked.nc")

In [104]:
dprec

<xarray.Dataset>
Dimensions:    (time: 14396, bnds: 2, lon: 177, lat: 138)
Coordinates:
  * time       (time) datetime64[ns] 1981-01-01T09:00:00 ... 2020-05-31T09:00:00
  * lon        (lon) float32 112.0 112.2 112.5 112.8 ... 155.2 155.5 155.8 156.0
  * lat        (lat) float32 -44.5 -44.25 -44.0 -43.75 ... -10.75 -10.5 -10.25
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    precip     (time, lat, lon) float32 ...
Attributes: (12/35)
    CDI:                       Climate Data Interface version 2.1.0 (https://...
    source:                    Data is from site-based observation data recor...
    institution:               Australian Bureau of Meteorology, GPO Box 1289...
    Conventions:               CF-1.6, ACDD-1.3
    geospatial_lat_min:        -44.525
    geospatial_lat_max:        -9.975
    ...                        ...
    description:               This AGCD data is a snapshot of the operationa...
    date_issued:               2020-12-02 05:42:26
    attribution:               Data should be cited as : Australian Bureau of...
    copyright:                 (C) Copyright Commonwealth of Australia 2020, ...
    history:                   Mon Jul 10 21:07:57 2023: cdo mul /Users/fadhl...
    CDO:                       Climate Data Operators version 2.1.0 (https://...

In [105]:
dw_1 = dw_1.sel(time=slice("1981-01-01","2018-12-31"))
dw_2 = dw_2.sel(time=slice("1981-01-01","2018-12-31"))
dw_3 = dw_3.sel(time=slice("1981-01-01","2018-12-31"))
dw_4 = dw_4.sel(time=slice("1981-01-01","2018-12-31"))
dw_5 = dw_5.sel(time=slice("1981-01-01","2018-12-31"))

dprec = dprec.sel(time=slice("1981-01-01","2018-12-31"),
                  lon=slice(lonLs[code],lonRs[code]), 
                  lat=slice(latSs[code],latNs[code])
                 )

prec = dprec.precip
prec

<xarray.DataArray 'precip' (time: 13879, lat: 24, lon: 17)>
[5662632 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01T09:00:00 ... 2018-12-31T09:00:00
  * lon      (lon) float32 142.0 142.2 142.5 142.8 ... 145.2 145.5 145.8 146.0
  * lat      (lat) float32 -16.0 -15.75 -15.5 -15.25 ... -10.75 -10.5 -10.25
Attributes:
    standard_name:                 lwe_thickness_of_precipitation_amount
    long_name:                     Daily precipitation
    units:                         mm
    cell_methods:                  time: sum
    analysis_version_number:       3.01
    number_of_stations_reporting:  6600
    frequency:                     daily
    length_scale_for_analysis:     80.0
    coverage_content_type:         physicalMeasurement

In [106]:
weights = np.cos(np.deg2rad(prec.lat))
weights.name = "weights"

pr_weighted = prec.weighted(weights)
praave = pr_weighted.mean(("lon", "lat"))

praave

<xarray.DataArray 'precip' (time: 13879)>
array([ 8.605104,  8.801927,  5.96713 , ..., 11.181782, 22.59539 ,
       39.38802 ], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01T09:00:00 ... 2018-12-31T09:00:00

In [107]:
er_active     = xr.where(dw_1["amp"] >= 1, dw_1, np.nan).dropna(dim="time")
kelvin_active = xr.where(dw_2["amp"] >= 1, dw_2, np.nan).dropna(dim="time")
mrg_active    = xr.where(dw_3["amp"] >= 1, dw_3, np.nan).dropna(dim="time")
td_active     = xr.where(dw_4["amp"] >= 1, dw_4, np.nan).dropna(dim="time")
mjo_active    = xr.where(dw_5["amp"] >= 1, dw_5, np.nan).dropna(dim="time")


In [108]:
# er_active.to_dataframe().to_csv('er_act.csv')

In [109]:
tc   = pd.read_csv('/Users/fadhlilrizki/Documents/PhD/thesis/data/TC_dates.csv',
                   usecols=["0"], parse_dates=["0"])

tc_da = xr.DataArray(tc)
er_active         = xr.where(~er_active['time'].isin(tc_da), er_active, np.nan).dropna(dim='time')
mrg_active        = xr.where(~mrg_active['time'].isin(tc_da), mrg_active, np.nan).dropna(dim='time')
kelvin_active     = xr.where(~kelvin_active['time'].isin(tc_da), kelvin_active, np.nan).dropna(dim='time')
td_active         = xr.where(~td_active['time'].isin(tc_da), td_active, np.nan).dropna(dim='time')
mjo_active        = xr.where(~mjo_active['time'].isin(tc_da), mjo_active, np.nan).dropna(dim='time')

er_active

<xarray.Dataset>
Dimensions:  (time: 2080)
Coordinates:
  * time     (time) datetime64[ns] 1981-03-01 1981-03-02 ... 2018-05-09
Data variables:
    x        (time) float32 0.7357 0.9528 1.047 ... 0.7224 0.3687 -0.02076
    y        (time) float32 0.7333 0.4502 0.0409 ... -0.8386 -1.073 -1.031
    amp      (time) float32 1.039 1.054 1.048 1.077 ... 1.029 1.107 1.134 1.032
    phase    (time) float64 8.0 8.0 1.0 2.0 2.0 3.0 ... 8.0 1.0 2.0 2.0 3.0 3.0

In [110]:
er_size = er_active.groupby("phase").groups
key_to_value_lengths = {k:len(v) for k, v in er_size.items()}
key_to_value_lengths

{1.0: 274,
 2.0: 281,
 3.0: 258,
 4.0: 244,
 5.0: 282,
 6.0: 259,
 7.0: 238,
 8.0: 244}

In [111]:
er_comp  = {}
klv_comp = {}
mrg_comp = {}
td_comp  = {}
mjo_comp = {}

In [112]:
praave.quantile(0.95)
praave

<xarray.DataArray 'precip' (time: 13879)>
array([ 8.605104,  8.801927,  5.96713 , ..., 11.181782, 22.59539 ,
       39.38802 ], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01T09:00:00 ... 2018-12-31T09:00:00

In [113]:
prprob = xr.where(praave<praave.quantile(0.95), 0, 1)
prob_season = prprob.sel(time=is_seas(prprob['time.month'],seas)).mean().values

prob_season

array(0.06006865)

In [114]:
er_comp

{}

In [115]:
er_comp = composites(er_active,prprob)
er_comp_mean = dict((k, np.mean(v[1])/prob_season) for k, v in er_comp.items())

mrg_comp = composites(mrg_active,prprob)
mrg_comp_mean = dict((k, np.mean(v[1])/prob_season) for k, v in mrg_comp.items())

kelvin_comp = composites(kelvin_active,prprob)
kelvin_comp_mean = dict((k, np.mean(v[1])/prob_season) for k, v in kelvin_comp.items())

td_comp = composites(td_active,prprob)
td_comp_mean = dict((k, np.mean(v[1])/prob_season) for k, v in td_comp.items())

mjo_comp = composites(mjo_active,prprob)
mjo_comp_mean = dict((k, np.mean(v[1])/prob_season) for k, v in mjo_comp.items())

/var/folders/41/txq0cr4j1kl0vqjl56x7l60w0000gp/T/ipykernel_66480/430224524.py:4: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  time_phase = wave_ds["time"].astype("datetime64[D]")
/var/folders/41/txq0cr4j1kl0vqjl56x7l60w0000gp/T/ipykernel_66480/430224524.py:5: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values 

In [116]:
er_comp
mrg_comp_mean

{'1': 0.9748605748605749,
 '2': 1.2485714285714284,
 '3': 0.6994797919167668,
 '4': 0.8720181405895692,
 '5': 1.0857142857142856,
 '6': 0.9704890027759983,
 '7': 1.3349505840071878,
 '8': 1.0066002214839425}

In [117]:
# #placeholdering (blank wet/dry phases are filled with arbitrary values)
 
# er_wet         = placehold(er_wet, 8)
# kelvin_wet     = placehold(kelvin_wet, 8)
# mrg_wet        = placehold(mrg_wet, 8)
# td_wet         = placehold(td_wet, 8)
# mjo_wet        = placehold(mjo_wet, 8)

 
# er_dry         = placehold(er_dry, 8)
# kelvin_dry     = placehold(kelvin_dry, 8)
# mrg_dry        = placehold(mrg_dry, 8)
# td_dry         = placehold(td_dry, 8)
# mjo_dry        = placehold(mjo_dry, 8)

# er_wet

In [118]:
# for i in er_wet:
#     er_time=[v[0] for k,v in er_comp.items() if int(k)==i]
# #     print(k)
# len(er_time[0])

In [119]:
# #compositing
# for i in er_active.groupby("phase").groups:
#     n = i.astype('int32')
# #     print(n)
#     time  = xr.where((er_active["phase"]==n), er_active, np.nan).dropna(dim="time")
    
#     print(len(time.coords["time"].values))
    
#     dates = time_precip
#     er_comp['phase-'+str(n)]  = xr.where(dates.isin(time.coords["time"].values), praave, np.nan).dropna(dim="time").mean()
#     del(time)
    
# #     klv_date['phase-'+str(n)] = xr.where((kelvin_active["phase"]==n), kelvin_active, np.nan).dropna(dim="time")
# #     mrg_date['phase-'+str(n)] = xr.where((mrg_active["phase"]==n), mrg_active, np.nan).dropna(dim="time")
# #     td_date['phase-'+str(n)]  = xr.where((td_active["phase"]==n), td_active, np.nan).dropna(dim="time")
# #     mjo_date['phase-'+str(n)] = xr.where((mjo_active["phase"]==n), mjo_active, np.nan).dropna(dim="time")

# er_date
# #     er_active.coords["time"].values

In [120]:
er_wets     = [int(k) for k,v in er_comp_mean.items() if v>=1.1]
kelvin_wets = [int(k) for k,v in kelvin_comp_mean.items() if v>=1.1]
mrg_wets    = [int(k) for k,v in mrg_comp_mean.items() if v>=1.1]
td_wets     = [int(k) for k,v in td_comp_mean.items() if v>=1.1]
mjo_wets    = [int(k) for k,v in mjo_comp_mean.items() if v>=1.1]

er_drys     = [int(k) for k,v in er_comp_mean.items() if v<0.9]
kelvin_drys = [int(k) for k,v in kelvin_comp_mean.items() if v<0.9]
mrg_drys    = [int(k) for k,v in mrg_comp_mean.items() if v<0.9]
td_drys     = [int(k) for k,v in td_comp_mean.items() if v<0.9]
mjo_drys    = [int(k) for k,v in mjo_comp_mean.items() if v<0.9]

with open('phases_CY_'+seas+'.txt', 'w') as f:
    f.write(str(["er-wet",er_wets,"dry", er_drys]))
    f.write("\n"+str(["kelvin-wet",kelvin_wets,"dry", kelvin_drys]))
    f.write("\n"+str(["mrg-wet",mrg_wets,"dry", mrg_drys]))
    f.write("\n"+str(["td-wet",td_wets,"dry", td_drys]))
    f.write("\n"+str(["mjo-wet",mjo_wets,"dry", mjo_drys]))

In [121]:

print(str(["er-wet",er_wets,"dry", er_drys]))
print("\n"+str(["kelvin-wet",kelvin_wets,"dry", kelvin_drys]))
print("\n"+str(["mrg-wet",mrg_wets,"dry", mrg_drys]))
print("\n"+str(["td-wet",td_wets,"dry", td_drys]))
print("\n"+str(["mjo-wet",mjo_wets,"dry", mjo_drys]))

['er-wet', [2, 4, 5], 'dry', [1, 7, 8]]

['kelvin-wet', [8], 'dry', [1, 2, 3, 4, 5, 6, 7]]

['mrg-wet', [2, 7], 'dry', [3, 4]]

['td-wet', [2, 3, 4, 5, 7], 'dry', []]

['mjo-wet', [6, 7, 8], 'dry', [1, 2, 3, 4]]


In [123]:
if seas=='djf':
    er_wet        = xr.where((er_active["phase"]==6), er_active, np.nan).dropna(dim="time")
    kelvin_wet    = xr.where((kelvin_active["phase"]==8), kelvin_active, np.nan).dropna(dim="time")
    mrg_wet       = xr.where((mrg_active["phase"]==8), mrg_active, np.nan).dropna(dim="time")
    td_wet        = xr.where((td_active["phase"]==8) , td_active, np.nan).dropna(dim="time")
    mjo_wet       = xr.where(((mjo_active["phase"]>=5) & (mjo_active["phase"]<=7)), mjo_active, np.nan).dropna(dim="time")
elif seas=='mam':
    er_wet        = xr.where(((er_active["phase"]>=4) & (er_active["phase"]<=5)) | (er_active["phase"]==2), er_active, np.nan).dropna(dim="time")
    kelvin_wet    = xr.where((kelvin_active["phase"]==8), kelvin_active, np.nan).dropna(dim="time")
    mrg_wet       = xr.where((mrg_active["phase"]==2) | (mrg_active["phase"]==7), mrg_active, np.nan).dropna(dim="time")
    td_wet        = xr.where((td_active["phase"]>=2) & (td_active["phase"]<=5) | (td_active["phase"]==7), td_active, np.nan).dropna(dim="time")
    mjo_wet       = xr.where(((mjo_active["phase"]>=6) & (mjo_active["phase"]<=8)), mjo_active, np.nan).dropna(dim="time")

In [124]:
if seas=='djf':
    er_dry        = xr.where((er_active["phase"]<=3) | (er_active["phase"]==8) | (er_active["phase"]==7), er_active, np.nan).dropna(dim="time")
    kelvin_dry    = xr.where((kelvin_active["phase"]!=2) & (kelvin_active["phase"]!=8), kelvin_active, np.nan).dropna(dim="time")
    mrg_dry       = xr.where((mrg_active["phase"]>=1), mrg_active, np.nan).dropna(dim="time")
    td_dry        = xr.where((td_active["phase"]<=4) | (td_active["phase"]==7), td_active, np.nan).dropna(dim="time")
    mjo_dry       = xr.where((mjo_active["phase"]<=4) | (mjo_active["phase"]==8), mjo_active, np.nan).dropna(dim="time")
elif seas=='mam':
    er_dry        = xr.where((er_active["phase"]==7) | (er_active["phase"]==1) | (er_active["phase"]==8), er_active, np.nan).dropna(dim="time")
    kelvin_dry    = xr.where((kelvin_active["phase"]!=8), kelvin_active, np.nan).dropna(dim="time")
    mrg_dry       = xr.where((mrg_active["phase"]>=3) & (mrg_active["phase"]<=4), mrg_active, np.nan).dropna(dim="time")
    td_dry        = xr.where((td_active["phase"]==8), td_active, np.nan).dropna(dim="time")
    mjo_dry       = xr.where((mjo_active["phase"]<=4) , mjo_active, np.nan).dropna(dim="time")

In [125]:
er_wet

<xarray.Dataset>
Dimensions:  (time: 807)
Coordinates:
  * time     (time) datetime64[ns] 1981-03-04 1981-03-05 ... 2018-05-07
Data variables:
    x        (time) float32 0.9811 0.7402 -0.5555 1.469 ... 0.7381 0.9497 0.7224
    y        (time) float32 -0.4435 -0.9096 -1.055 ... -1.318 -0.3963 -0.8386
    amp      (time) float32 1.077 1.173 1.192 2.397 ... 1.895 1.511 1.029 1.107
    phase    (time) float64 2.0 2.0 4.0 2.0 4.0 4.0 ... 5.0 2.0 2.0 2.0 2.0 2.0

In [126]:
er_wet

<xarray.Dataset>
Dimensions:  (time: 807)
Coordinates:
  * time     (time) datetime64[ns] 1981-03-04 1981-03-05 ... 2018-05-07
Data variables:
    x        (time) float32 0.9811 0.7402 -0.5555 1.469 ... 0.7381 0.9497 0.7224
    y        (time) float32 -0.4435 -0.9096 -1.055 ... -1.318 -0.3963 -0.8386
    amp      (time) float32 1.077 1.173 1.192 2.397 ... 1.895 1.511 1.029 1.107
    phase    (time) float64 2.0 2.0 4.0 2.0 4.0 4.0 ... 5.0 2.0 2.0 2.0 2.0 2.0

In [127]:
#ER AND KELVIN
er_er_wetwet  = er_wet["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")
er_er_drydry  = er_dry["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")

klv_klv_wetwet  = kelvin_wet["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")
klv_klv_drydry  = kelvin_dry["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")

mrg_mrg_wetwet  = mrg_wet["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_mrg_drydry  = mrg_dry["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")

td_td_wetwet  = td_wet["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")
td_td_drydry  = td_dry["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")

mjo_mjo_wetwet  = mjo_wet["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")
mjo_mjo_drydry  = mjo_dry["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")


#ER kelvin
er_klv_wetwet = kelvin_wet["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_klv_drywet = kelvin_wet["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_klv_drydry = kelvin_dry["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_klv_wetdry = kelvin_dry["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

#MRG and KELVIN

mrg_klv_wetwet = kelvin_wet["time"].where(mrg_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_klv_drywet = kelvin_wet["time"].where(mrg_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_klv_drydry = kelvin_dry["time"].where(mrg_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_klv_wetdry = kelvin_dry["time"].where(mrg_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

#TD and Kelvin

td_klv_wetwet  = kelvin_wet["time"].where(td_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
td_klv_drywet  = kelvin_wet["time"].where(td_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
td_klv_drydry  = kelvin_dry["time"].where(td_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
td_klv_wetdry  = kelvin_dry["time"].where(td_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

#MJO and Kelvin

mjo_klv_wetwet  = kelvin_wet["time"].where(mjo_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mjo_klv_drywet  = kelvin_wet["time"].where(mjo_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mjo_klv_drydry  = kelvin_dry["time"].where(mjo_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mjo_klv_wetdry  = kelvin_dry["time"].where(mjo_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")



In [128]:
#ER AND MRG
er_mrg_wetwet = mrg_wet["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_mrg_drywet = mrg_wet["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_mrg_drydry = mrg_dry["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_mrg_wetdry = mrg_dry["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

In [129]:
#ER AND TD
er_td_wetwet = td_wet["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_td_drywet = td_wet["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_td_drydry = td_dry["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_td_wetdry = td_dry["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

In [130]:
#ER AND MJO
er_mjo_wetwet = mjo_wet["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_mjo_drywet = mjo_wet["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_mjo_drydry = mjo_dry["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_mjo_wetdry = mjo_dry["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

In [131]:
#mrg AND MJO
mrg_mjo_wetwet = mjo_wet["time"].where(mrg_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_mjo_drywet = mjo_wet["time"].where(mrg_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_mjo_drydry = mjo_dry["time"].where(mrg_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_mjo_wetdry = mjo_dry["time"].where(mrg_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

In [132]:
#TD AND MJO
td_mjo_wetwet = mjo_wet["time"].where(td_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
td_mjo_drywet = mjo_wet["time"].where(td_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
td_mjo_drydry = mjo_dry["time"].where(td_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
td_mjo_wetdry = mjo_dry["time"].where(td_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

In [133]:
#MRG AND TD
mrg_td_wetwet = td_wet["time"].where(mrg_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_td_drywet = td_wet["time"].where(mrg_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_td_drydry = td_dry["time"].where(mrg_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_td_wetdry = td_dry["time"].where(mrg_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

In [134]:
er_er_wetwet

,date
time,
1981-03-04,19810304
1981-03-05,19810305
1981-03-08,19810308
1981-03-18,19810318
1981-03-21,19810321
...,...
2018-04-26,20180426
2018-04-27,20180427
2018-04-28,20180428


In [135]:
er_er_wetwet.to_csv(diro+"er_er_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
er_er_drydry.to_csv(diro+"er_er_drydry_CY_"+seas+".csv", columns=['date'], index = False)

er_klv_wetwet.to_csv(diro+"er_klv_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
er_klv_wetdry.to_csv(diro+"er_klv_wetdry_CY_"+seas+".csv", columns=['date'], index = False)
er_klv_drywet.to_csv(diro+"er_klv_drywet_CY_"+seas+".csv", columns=['date'], index = False)
er_klv_drydry.to_csv(diro+"er_klv_drydry_CY_"+seas+".csv", columns=['date'], index = False)

er_mrg_wetwet.to_csv(diro+"er_mrg_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
er_mrg_wetdry.to_csv(diro+"er_mrg_wetdry_CY_"+seas+".csv", columns=['date'], index = False)
er_mrg_drywet.to_csv(diro+"er_mrg_drywet_CY_"+seas+".csv", columns=['date'], index = False)
er_mrg_drydry.to_csv(diro+"er_mrg_drydry_CY_"+seas+".csv", columns=['date'], index = False)

er_td_wetwet.to_csv(diro+"er_td_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
er_td_wetdry.to_csv(diro+"er_td_wetdry_CY_"+seas+".csv", columns=['date'], index = False)
er_td_drywet.to_csv(diro+"er_td_drywet_CY_"+seas+".csv", columns=['date'], index = False)
er_td_drydry.to_csv(diro+"er_td_drydry_CY_"+seas+".csv", columns=['date'], index = False)

er_mjo_wetwet.to_csv(diro+"er_mjo_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
er_mjo_wetdry.to_csv(diro+"er_mjo_wetdry_CY_"+seas+".csv", columns=['date'], index = False)
er_mjo_drywet.to_csv(diro+"er_mjo_drywet_CY_"+seas+".csv", columns=['date'], index = False)
er_mjo_drydry.to_csv(diro+"er_mjo_drydry_CY_"+seas+".csv", columns=['date'], index = False)

In [136]:
klv_klv_wetwet.to_csv(diro+"klv_klv_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
klv_klv_drydry.to_csv(diro+"klv_klv_drydry_CY_"+seas+".csv", columns=['date'], index = False)

mrg_mrg_wetwet.to_csv(diro+"mrg_mrg_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
mrg_mrg_drydry.to_csv(diro+"mrg_mrg_drydry_CY_"+seas+".csv", columns=['date'], index = False)

mrg_klv_wetwet.to_csv(diro+"mrg_klv_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
mrg_klv_wetdry.to_csv(diro+"mrg_klv_wetdry_CY_"+seas+".csv", columns=['date'], index = False)
mrg_klv_drywet.to_csv(diro+"mrg_klv_drywet_CY_"+seas+".csv", columns=['date'], index = False)
mrg_klv_drydry.to_csv(diro+"mrg_klv_drydry_CY_"+seas+".csv", columns=['date'], index = False)

mrg_td_wetwet.to_csv(diro+"mrg_td_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
mrg_td_wetdry.to_csv(diro+"mrg_td_wetdry_CY_"+seas+".csv", columns=['date'], index = False)
mrg_td_drywet.to_csv(diro+"mrg_td_drywet_CY_"+seas+".csv", columns=['date'], index = False)
mrg_td_drydry.to_csv(diro+"mrg_td_drydry_CY_"+seas+".csv", columns=['date'], index = False)

mrg_mjo_wetwet.to_csv(diro+"mrg_mjo_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
mrg_mjo_wetdry.to_csv(diro+"mrg_mjo_wetdry_CY_"+seas+".csv", columns=['date'], index = False)
mrg_mjo_drywet.to_csv(diro+"mrg_mjo_drywet_CY_"+seas+".csv", columns=['date'], index = False)
mrg_mjo_drydry.to_csv(diro+"mrg_mjo_drydry_CY_"+seas+".csv", columns=['date'], index = False)

In [137]:
td_td_wetwet.to_csv(diro+"td_td_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
td_td_drydry.to_csv(diro+"td_td_drydry_CY_"+seas+".csv", columns=['date'], index = False)

td_klv_wetwet.to_csv(diro+"td_klv_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
td_klv_wetdry.to_csv(diro+"td_klv_wetdry_CY_"+seas+".csv", columns=['date'], index = False)
td_klv_drywet.to_csv(diro+"td_klv_drywet_CY_"+seas+".csv", columns=['date'], index = False)
td_klv_drydry.to_csv(diro+"td_klv_drydry_CY_"+seas+".csv", columns=['date'], index = False)

td_mjo_wetwet.to_csv(diro+"td_mjo_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
td_mjo_wetdry.to_csv(diro+"td_mjo_wetdry_CY_"+seas+".csv", columns=['date'], index = False)
td_mjo_drywet.to_csv(diro+"td_mjo_drywet_CY_"+seas+".csv", columns=['date'], index = False)
td_mjo_drydry.to_csv(diro+"td_mjo_drydry_CY_"+seas+".csv", columns=['date'], index = False)


In [138]:

mjo_mjo_wetwet.to_csv(diro+"mjo_mjo_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
mjo_mjo_drydry.to_csv(diro+"mjo_mjo_drydry_CY_"+seas+".csv", columns=['date'], index = False)

mjo_klv_wetwet.to_csv(diro+"mjo_klv_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
mjo_klv_wetdry.to_csv(diro+"mjo_klv_wetdry_CY_"+seas+".csv", columns=['date'], index = False)
mjo_klv_drywet.to_csv(diro+"mjo_klv_drywet_CY_"+seas+".csv", columns=['date'], index = False)
mjo_klv_drydry.to_csv(diro+"mjo_klv_drydry_CY_"+seas+".csv", columns=['date'], index = False)


In [139]:
er_td_drydry

,date
time,
1981-05-09,19810509
1982-03-08,19820308
1983-03-16,19830316
1983-05-09,19830509
1985-04-08,19850408
1985-04-25,19850425
1986-03-21,19860321
1987-03-06,19870306
1988-03-04,19880304


In [140]:
! ncl 'seas="{seas}"' code=1 composite_combined_area_wave_forpy.ncl

 Copyright (C) 1995-2019 - All Rights Reserved
 University Corporation for Atmospheric Research
 NCAR Command Language Version 6.6.2
 The use of this software is governed by a License Agreement.
 See http://www.ncl.ucar.edu/ for more details.

Variable: pr
Type: float
Total Size: 57584 bytes
            14396 values
Number of Dimensions: 1
Dimensions and sizes:	[time | 14396]
Coordinates: 
            time: [47847.375..62242.375]
Number Of Attributes: 13
  _FillValue :	1e+20
  coverage_content_type :	physicalMeasurement
  length_scale_for_analysis :	  80
  frequency :	daily
  number_of_stations_reporting :	6600
  analysis_version_number :	3.01
  cell_methods :	time: sum
  least_significant_digit :	3
  units :	mm
  long_name :	Daily precipitation
  standard_name :	lwe_thickness_of_precipitation_amount
  missing_value :	1e+20
  wgt_areaave_op_ncl :	Area Average


Variable: pr_Anom
Type: float
Total Size: 57584 bytes
            14396 values
Number of Dimensions: 1
Dimensions and sizes:	[

In [141]:
! ncl 'seas="{seas}"' code=1 composite_combined_area_wave_extremes_forpy.ncl

 Copyright (C) 1995-2019 - All Rights Reserved
 University Corporation for Atmospheric Research
 NCAR Command Language Version 6.6.2
 The use of this software is governed by a License Agreement.
 See http://www.ncl.ucar.edu/ for more details.

Variable: pr_aave
Type: float
Total Size: 57584 bytes
            14396 values
Number of Dimensions: 1
Dimensions and sizes:	[time | 14396]
Coordinates: 
            time: [47847.375..62242.375]
Number Of Attributes: 13
  _FillValue :	1e+20
  coverage_content_type :	physicalMeasurement
  length_scale_for_analysis :	  80
  frequency :	daily
  number_of_stations_reporting :	6600
  analysis_version_number :	3.01
  cell_methods :	time: sum
  least_significant_digit :	3
  units :	mm
  long_name :	Daily precipitation
  standard_name :	lwe_thickness_of_precipitation_amount
  missing_value :	1e+20
  wgt_areaave_op_ncl :	Area Average


Variable: pr_stat
Type: float
Total Size: 120 bytes
            30 values
Number of Dimensions: 1
Dimensions and sizes:	[